<a href="https://colab.research.google.com/github/emmanuelvaie/google_colab/blob/main/proto_dame_de_nages.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Get references data
Find Mattieu's file and read it into a dataframe


In [1]:
# piece of code that get a token
from pydrive.auth import GoogleAuth
from pydrive.drive import GoogleDrive
from google.colab import auth
from oauth2client.client import GoogleCredentials

# authenticate the user
auth.authenticate_user()
gauth = GoogleAuth()
gauth.credentials = GoogleCredentials.get_application_default()
drive = GoogleDrive(gauth)


Once authenticated, list the files that contains personal records for the atheletes


In [2]:
# Auto-iterate through all files that matches this query
#file_list = drive.ListFile({'q': "'root' in parents and title contains 'Champs' and trashed=false"}).GetList()
#sharedWithMe=true 
# id = 1VJMXVNSxIv4li8yK7pXZ3sitWEdC8z-93z_Apkl77_Q
file_list = drive.ListFile({'q': "trashed=false and title contains '2022.05.15-Nico - '"}).GetList()
for file1 in file_list:
  print('title: %s, id: %s' % (file1['title'], file1['id']))

title: 2022.05.15-Nico - 3.csv, id: 1a-OP8ilFEVL_bCioayaGJGDi1fh7M-Ol
title: 2022.05.15-Nico - 2.csv, id: 12kZ98a6IHGCWbQfguBA0Cb56CL2-REHL
title: 2022.05.15-Nico - 1.csv, id: 1E0nwCTgYgxOqU-tXJ3MludLZe4zF4f3Q


once the id of the spreadsheet retrieved, load it. 
But first, we need to be allowed Gspread  / GSheet to access to our data.

In [5]:
from google.colab import auth
auth.authenticate_user()

import gspread
from google.auth import default
creds, _ = default()

gc = gspread.authorize(creds)

# Get the data from the outriggers

In [37]:
import pandas as pd

# we have the list
ddd_list = list()
for ddd_file in file_list:
  # collect meta data from the filename
  filename = ddd_file['title']
  file_id = ddd_file['id']
  tokens = filename.split('-')
  rower = tokens[1].strip().lower()
  date = tokens[0].split('.')[2].strip() + '/' + tokens[0].split('.')[1].strip() + '/' + tokens[0].split('.')[0].strip()
  training_type = 'tour-'+ tokens[2].strip() 
  print('rower: %s, training type: %s, date: %s' % (rower, training_type, date))
  
  # this assume pyDrive has a token to read the GoogleDrive
  myfile = drive.CreateFile({'id': file_id})
  myfile.GetContentFile('file.csv')
  
  # then read it in a Dataframe
  ddd = pd.read_csv('file.csv', skiprows=18, header=None)

  # clean and organize the data
  # column names un 1st row
  # reformat
  headers = ddd.iloc[0].apply(lambda x: str(x).replace(' ', '_'))
  headers = headers.apply(lambda x: str(x).replace('(', ''))
  headers = headers.apply(lambda x: str(x).replace(')', ''))
  headers = headers.apply(lambda x: str(x).replace('.', ''))
  headers = headers.apply(lambda x: str(x).lower())

  # rename columns
  ddd = ddd.rename(columns=headers)

  # dropping NA values
  ddd2 = ddd.dropna()

  # drop the 2 first lines
  ddd2 = ddd2.drop([0, 1])

  # drop the 2 last rows
  #ddd2 = ddd2.drop(ddd2.tail(2).index)

  # reset the index
  ddd2 = ddd2.reset_index(drop=True)

  # changing column types
  column_names = list(ddd2.columns)

  # datetime columns
  # remove lines with date time columns formated to '---'
  ddd2 = ddd2.drop(ddd2[ddd2['elapsed_time'].map(lambda x: str(x) == '---')].index)
  ddd2 = ddd2.drop(ddd2[ddd2['split_gps'].map(lambda x: str(x) == '---')].index)
  ddd2 = ddd2.drop(ddd2[ddd2['split_imp'].map(lambda x: str(x) == '---')].index)

  # iterating on columns and change the types to string
  for col in column_names:
      ddd2[col] = ddd2[col].apply(lambda x: str(x).replace('---', '0'))

  ddd2 = ddd2.convert_dtypes()

  # removing datetime columns from the columns list
  column_names.pop(3)  # 'elapsed_time'
  column_names.pop(3)  # 'split_gps'
  column_names.pop(4)  # 'split_imp'

  # add meta data
  ddd2['rower'] = rower
  ddd2['training_type'] = training_type
  ddd2['date'] = date

  for col in column_names:
    ddd2[col] = pd.to_numeric(ddd2[col])

  # add columns amplitude and amplitude_efficace
  ddd2['amplitude'] = -1*(ddd2['catch'] - ddd2['finish'])
  ddd2['amplitude_efficace'] = - ddd2['catch'] - ddd2['slip'] + ddd2['finish'] - ddd2['wash']

  # add loc in google format
  ddd2['loc'] = ddd2['gps_lat'].apply(lambda x: str(x)) + ',' + ddd2['gps_lon'].apply(lambda x: str(x))

 
  # accumulate ddd
  ddd_list.append(ddd2)

# concat dataframes
all_ddd = pd.concat(ddd_list)


rower: nico, training type: tour-3.csv, date: 15/05/2022
rower: nico, training type: tour-2.csv, date: 15/05/2022
rower: nico, training type: tour-1.csv, date: 15/05/2022


In [38]:
all_ddd.head(20)

,interval,distance_gps,distance_imp,elapsed_time,split_gps,speed_gps,split_imp,speed_imp,stroke_rate,total_strokes,...,force_max,max_force_angle,gps_lat,gps_lon,rower,training_type,date,amplitude,amplitude_efficace,loc
0,1,14740.8,0.0,01:24:28.1,00:03:01.1,2.76,00:00:00.0,0.0,18.5,1448,...,353,-11,48.899211,2.277566,nico,tour-3.csv,15/05/2022,109,92,"48.8992109,2.2775662"
1,1,14750.8,0.0,01:24:31.4,00:02:50.0,2.94,00:00:00.0,0.0,18.0,1449,...,378,-1,48.899162,2.277452,nico,tour-3.csv,15/05/2022,107,85,"48.8991619,2.277452"
2,1,14760.6,0.0,01:24:34.8,00:02:48.3,2.97,00:00:00.0,0.0,18.0,1450,...,368,-6,48.899116,2.277337,nico,tour-3.csv,15/05/2022,106,84,"48.8991163,2.2773369"
3,1,14769.7,0.0,01:24:37.8,00:02:48.9,2.96,00:00:00.0,0.0,19.0,1451,...,333,5,48.899074,2.277231,nico,tour-3.csv,15/05/2022,103,82,"48.8990738,2.2772309"
4,1,14779.5,0.0,01:24:41.2,00:02:48.3,2.97,00:00:00.0,0.0,18.0,1452,...,361,-9,48.899028,2.277116,nico,tour-3.csv,15/05/2022,107,91,"48.8990276,2.2771162"
5,1,14789.6,0.0,01:24:44.6,00:02:50.0,2.94,00:00:00.0,0.0,18.0,1453,...,364,-6,48.898980,2.277000,nico,tour-3.csv,15/05/2022,106,86,"48.8989797,2.2769995"
6,1,14799.7,0.0,01:24:48.0,00:02:50.6,2.93,00:00:00.0,0.0,17.0,1454,...,351,-7,48.898931,2.276883,nico,tour-3.csv,15/05/2022,107,86,"48.8989312,2.2768826"
7,1,14810.5,0.0,01:24:51.9,00:02:50.6,2.93,00:00:00.0,0.0,17.0,1455,...,337,-10,48.898879,2.276758,nico,tour-3.csv,15/05/2022,107,91,"48.8988795,2.2767581"
8,1,14820.4,0.0,01:24:55.0,00:02:50.6,2.93,00:00:00.0,0.0,17.5,1456,...,390,0,48.898830,2.276645,nico,tour-3.csv,15/05/2022,106,92,"48.8988302,2.2766449"
9,1,14830.1,0.0,01:24:58.1,00:02:46.1,3.01,00:00:00.0,0.0,18.5,1457,...,353,-9,48.898787,2.276531,nico,tour-3.csv,15/05/2022,107,91,"48.8987869,2.2765309"


In [39]:
all_ddd.dtypes

interval                 int64
distance_gps           float64
distance_imp           float64
elapsed_time            string
split_gps               string
speed_gps              float64
split_imp               string
speed_imp              float64
stroke_rate            float64
total_strokes            int64
distance/stroke_gps    float64
distance/stroke_imp    float64
heart_rate               int64
power                    int64
catch                    int64
slip                     int64
finish                   int64
wash                     int64
force_avg                int64
work                     int64
force_max                int64
max_force_angle          int64
gps_lat                float64
gps_lon                float64
rower                   object
training_type           object
date                    object
amplitude                int64
amplitude_efficace       int64
loc                     object
dtype: object

Adding a workseet with the dame de nage data

In [40]:
import gspread
import gspread_dataframe as gd

db_sh = gc.open_by_key('1u4KNFewcNQ7YpfTW0L1S2RM6Wu87xar-5zFzvXa2T2o')

worksheet_data = db_sh.worksheet('database')
gd.set_with_dataframe(worksheet_data,all_ddd, include_index=False,include_column_header=True,resize=True)